# EventVestor: Impairments and Charges

In this notebook, we'll take a look at EventVestor's *Impairments and Charges* dataset, available on the [Quantopian Store](https://www.quantopian.com/store). This dataset spans January 01, 2007 through the current day, and documents goodwill impairments and other one time charges reported by companies.

### Blaze
Before we dig into the data, we want to tell you about how  you generally access Quantopian Store data sets. These datasets are available through an API service known as [Blaze](http://blaze.pydata.org). Blaze provides the Quantopian user with a convenient interface to access very large datasets.

Blaze provides an important function for accessing these datasets. Some of these sets are many millions of records. Bringing that data directly into Quantopian Research directly just is not viable. So Blaze allows us to provide a simple querying interface and shift the burden over to the server side.

It is common to use Blaze to reduce your dataset in size, convert it over to Pandas and then to use Pandas for further computation, manipulation and visualization.

Helpful links:
* [Query building for Blaze](http://blaze.pydata.org/en/latest/queries.html)
* [Pandas-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-pandas.html)
* [SQL-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-sql.html).

Once you've limited the size of your Blaze object, you can convert it to a Pandas DataFrames using:
> `from odo import odo`  
> `odo(expr, pandas.DataFrame)`

### Free samples and limits
One other key caveat: we limit the number of results returned from any given expression to 10,000 to protect against runaway memory usage. To be clear, you have access to all the data server side. We are limiting the size of the responses back from Blaze.

There is a *free* version of this dataset as well as a paid one. The free one includes about three years of historical data, though not up to the current day.

With preamble in place, let's get started:

In [1]:
# import the dataset
from quantopian.interactive.data.eventvestor import impairments_and_charges
# or if you want to import the free dataset, use:
# from quantopian.interactive.data.eventvestor import impairments_and_charges_free

# import data operations
from odo import odo
# import other libraries we will use
import pandas as pd

In [2]:
# Let's use blaze to understand the data a bit using Blaze dshape()
impairments_and_charges.dshape

dshape("""var * {
  event_id: ?float64,
  asof_date: datetime,
  trade_date: ?datetime,
  symbol: ?string,
  event_type: ?string,
  event_headline: ?string,
  charge_amount: ?float64,
  amount_units: ?string,
  event_rating: ?float64,
  timestamp: datetime,
  sid: ?int64
  }""")

In [3]:
# And how many rows are there?
# N.B. we're using a Blaze function to do this, not len()
impairments_and_charges.count()

3991

In [4]:
# Let's see what the data looks like. We'll grab the first three rows.
impairments_and_charges[:3]

,event_id,asof_date,trade_date,symbol,event_type,event_headline,charge_amount,amount_units,event_rating,timestamp,sid
0,131321,2007-01-05,2007-01-08,GT,Impairments/Charges,Goodyear To Record $155M To $160M Charges in 1...,160,$M,1,2007-01-06,3384
1,110962,2007-01-08,2007-01-09,MO,Impairments/Charges,Altria Group Subsidiary To Record $245M Asset ...,245,$M,1,2007-01-09,4954
2,1182869,2007-01-16,2007-01-16,FRX,Impairments/Charges,Forest Labs to Record $494M Charge in 4Q 07,494,$M,1,2007-01-17,3014


Let's go over the columns:
- **event_id**: the unique identifier for this event.
- **asof_date**: EventVestor's timestamp of event capture.
- **trade_date**: for event announcements made before trading ends, trade_date is the same as event_date. For announcements issued after market close, trade_date is next market open day.
- **symbol**: stock ticker symbol of the affected company.
- **event_type**: this should always be *Impairments/Charges*.
- **event_headline**: a brief description of the event
- **charge_amount**: amount charged in `amount_units`
- **amount_units**: units of the amount charged. Most commonly millions of dollars.
- **event_rating**: this is always 1. The meaning of this is uncertain.
- **timestamp**: this is our timestamp on when we registered the data.
- **sid**: the equity's unique identifier. Use this instead of the symbol.

We've done much of the data processing for you. Fields like `timestamp` and `sid` are standardized across all our Store Datasets, so the datasets are easy to combine. We have standardized the `sid` across all our equity databases.

We can select columns and rows with ease. Below, we'll fetch all 2012 charges greater than $200M.

In [5]:
twohundreds = impairments_and_charges[('2011-12-31' < impairments_and_charges['asof_date']) & 
                                        (impairments_and_charges['asof_date'] <'2013-01-01') & 
                                        (impairments_and_charges.charge_amount > 200)&
                                        (impairments_and_charges.amount_units == "$M")]
# When displaying a Blaze Data Object, the printout is automatically truncated to ten rows.
twohundreds.sort('asof_date')

,event_id,asof_date,trade_date,symbol,event_type,event_headline,charge_amount,amount_units,event_rating,timestamp,sid
0,1382496,2012-01-11,2012-01-12,XL,Impairments/Charges,XL Group to Record Upto $220M Charges,220,$M,1,2012-01-12,8340
1,1382455,2012-01-11,2012-01-12,RF,Impairments/Charges,Regions Financial to Record Upto $745M Impairm...,745,$M,1,2012-01-12,34913
2,1383159,2012-01-13,2012-01-16,ADM,Impairments/Charges,Archer Daniels to Record Upto $360M Charge in ...,360,$M,1,2012-01-14,128
3,1383004,2012-01-13,2012-01-13,NVS,Impairments/Charges,Novartis to Record $1.22B Charges,1220,$M,1,2012-01-14,21536
4,1383880,2012-01-18,2012-01-18,HES,Impairments/Charges,Hess Corp to Record $525M Charge in 4Q 11 on R...,525,$M,1,2012-01-19,216
5,1384387,2012-01-19,2012-01-19,ECL,Impairments/Charges,Ecolab to Record $480M Charges by FY 13,480,$M,1,2012-01-20,2427
6,1385496,2012-01-23,2012-01-24,MUR,Impairments/Charges,Murphy Oil Unit to Record $370M Asset Impairme...,370,$M,1,2012-01-24,5126
7,1386032,2012-01-24,2012-01-25,BBOX,Impairments/Charges,Black Box to Record $320M Charges in 3Q 12,320,$M,1,2012-01-25,11732
8,1385962,2012-01-24,2012-01-25,RE,Impairments/Charges,Everest Re Group to Record $245M Catastrophe L...,245,$M,1,2012-01-25,13720
9,1388133,2012-01-30,2012-01-30,X,Impairments/Charges,United States Steel to Record Upto $450M Charg...,450,$M,1,2012-01-31,8329


Now suppose we want a DataFrame of the Blaze Data Object above, and we only want the sid, charge_amount and the asof_date.

In [6]:
df = odo(twohundreds, pd.DataFrame)
df = df[['sid', 'asof_date','charge_amount']].dropna()
# When printing a pandas DataFrame, the head 30 and tail 30 rows are displayed. The middle is truncated.
df

,sid,asof_date,charge_amount
0,34913,2012-01-11,745.0
1,8340,2012-01-11,220.0
2,128,2012-01-13,360.0
3,21536,2012-01-13,1220.0
4,216,2012-01-18,525.0
5,2427,2012-01-19,480.0
6,5126,2012-01-23,370.0
7,11732,2012-01-24,320.0
8,13720,2012-01-24,245.0
9,8329,2012-01-30,450.0
